In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
traindata = pd.read_csv('Train.csv')
riders = pd.read_csv('Riders.csv')
testdata = pd.read_csv('Test.csv')

In [64]:
# train = traindata.merge(riders, on='Rider Id')

# functions for pre-processing

In [65]:
def time_from_midnight_in_seconds(dataset, column):
    dataset[column] = pd.to_datetime(dataset[column])
    return (dataset[column] - pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')

In [66]:
def delta_time(dataset, higher_time, lower_time):
    return dataset[higher_time] - dataset[lower_time]

In [67]:
def average(list_data):
    return sum(list_data)/len(list_data)+1

In [68]:
def time_to_day_part(time):
    hours = time/3600
    if hours < 6:
        return 'night'
    if hours < 12:
        return 'morning'
    if hours < 18:
        return 'afternoon'
    else:
        return 'evening'

In [69]:
def calculate_bearing(lat1, lng1, lat2, lng2):
    lat1 = np.deg2rad(lat1)
    lat2 = np.deg2rad(lat2)
    diffLong = np.deg2rad(lng2 - lng1)
    x = np.sin(diffLong) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1)
                                       * np.cos(lat2) * np.cos(diffLong))
    initial_bearing = np.arctan2(x, y)
     # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = np.rad2deg(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360
    return compass_bearing

In [70]:
traindata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,...,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,...,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,...,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [71]:
# converting time in seconds from midnight for train data
traindata['Placement - Time'] = time_from_midnight_in_seconds(traindata, 'Placement - Time')
traindata['Confirmation - Time'] = time_from_midnight_in_seconds(traindata, 'Confirmation - Time')
traindata['Pickup - Time'] = time_from_midnight_in_seconds(traindata, 'Pickup - Time')
traindata['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(traindata, 'Arrival at Pickup - Time')
traindata['Arrival at Destination - Time'] = time_from_midnight_in_seconds(traindata, 'Arrival at Destination - Time')

In [72]:
traindata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,38395.0,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,40576.0,12,5,...,44242.0,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,45565.0,30,2,...,46838.0,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,33934.0,15,5,...,36327.0,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,35718.0,13,1,...,37537.0,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [73]:
testdata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,...,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,...,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,...,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,...,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,...,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [74]:
# converting time in seconds from midnight for test data
testdata['Placement - Time'] = time_from_midnight_in_seconds(testdata, 'Placement - Time')
testdata['Confirmation - Time'] = time_from_midnight_in_seconds(testdata, 'Confirmation - Time')
testdata['Pickup - Time'] = time_from_midnight_in_seconds(testdata, 'Pickup - Time')
testdata['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(testdata, 'Arrival at Pickup - Time')

In [75]:
testdata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,60250.0,27,3,...,3,61607.0,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,46655.0,17,5,...,5,48337.0,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,40094.0,27,4,...,4,43074.0,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,49895.0,17,1,...,1,51412.0,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,41428.0,11,2,...,2,42964.0,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [76]:
# calculating delta_time for train data
delta_confirm_place_train = delta_time(traindata, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_train = delta_time(traindata, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_train = delta_time(traindata, 'Pickup - Time', 'Arrival at Pickup - Time')
delta_arrival_pickup_train = delta_time(traindata, 'Arrival at Destination - Time', 'Pickup - Time')
delta_placement_arrival_train = delta_time(traindata, 'Arrival at Destination - Time', 'Placement - Time')

In [77]:
traindata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,38395.0,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,40576.0,12,5,...,44242.0,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,45565.0,30,2,...,46838.0,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,33934.0,15,5,...,36327.0,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,35718.0,13,1,...,37537.0,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [78]:
# calculating delta_time for test data
delta_confirm_place_test = delta_time(testdata, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_test = delta_time(testdata, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_test = delta_time(testdata, 'Pickup - Time', 'Arrival at Pickup - Time')

In [79]:
# merging riders data and train data
ridertrain = traindata.merge(riders, on='Rider Id')
ridertest = testdata.merge(riders, on='Rider Id')

In [80]:
ridertrain.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549
1,Order_No_27440,User_Id_2642,Bike,3,Personal,18,5,56477.0,18,5,...,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549
2,Order_No_14170,User_Id_3339,Bike,3,Business,31,5,46301.0,31,5,...,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549
3,Order_No_21727,User_Id_3523,Bike,3,Personal,2,2,25930.0,2,2,...,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549
4,Order_No_8301,User_Id_87,Bike,2,Personal,22,2,38458.0,22,2,...,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549


In [81]:
ridertest.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,60250.0,27,3,...,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,439,1511,13.3,171
1,Order_No_2699,User_Id_592,Bike,1,Personal,26,2,53001.0,26,2,...,NaN,-1.312745,36.875715,-1.288435,36.760677,Rider_Id_192,439,1511,13.3,171
2,Order_No_21486,User_Id_478,Bike,3,Business,29,5,42905.0,29,5,...,NaN,-1.255189,36.782203,-1.293393,36.790237,Rider_Id_192,439,1511,13.3,171
3,Order_No_19336,User_Id_2801,Bike,3,Business,2,1,33801.0,2,1,...,NaN,-1.310451,36.868724,-1.259542,36.787118,Rider_Id_192,439,1511,13.3,171
4,Order_No_20374,User_Id_3465,Bike,3,Business,14,4,46559.0,14,4,...,NaN,-1.333275,36.870815,-1.255189,36.782203,Rider_Id_192,439,1511,13.3,171


In [82]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
Order No                                     21201 non-null object
User Id                                      21201 non-null object
Vehicle Type                                 21201 non-null object
Platform Type                                21201 non-null int64
Personal or Business                         21201 non-null object
Placement - Day of Month                     21201 non-null int64
Placement - Weekday (Mo = 1)                 21201 non-null int64
Placement - Time                             21201 non-null float64
Confirmation - Day of Month                  21201 non-null int64
Confirmation - Weekday (Mo = 1)              21201 non-null int64
Confirmation - Time                          21201 non-null float64
Arrival at Pickup - Day of Month             21201 non-null int64
Arrival at Pickup - Weekday (Mo = 1)         21201 non-null int64
Arrival at Pickup - Time 

In [83]:
# Treat Outliers in Training Data
ridertrain = ridertrain[ridertrain['Placement - Weekday (Mo = 1)'] == ridertrain['Confirmation - Weekday (Mo = 1)']]
ridertrain = ridertrain[ridertrain['Placement - Day of Month'] == ridertrain['Confirmation - Day of Month']]

In [84]:
# input missing values for Temperature  for train and test data
ridertrain['Temperature'].fillna(ridertrain['Temperature'].mean(), inplace=True)
ridertest['Temperature'].fillna(ridertest['Temperature'].mean(), inplace=True)

# pre-processing

In [85]:
ridertrain.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549
1,Order_No_27440,User_Id_2642,Bike,3,Personal,18,5,56477.0,18,5,...,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549
2,Order_No_14170,User_Id_3339,Bike,3,Business,31,5,46301.0,31,5,...,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549
3,Order_No_21727,User_Id_3523,Bike,3,Personal,2,2,25930.0,2,2,...,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549
4,Order_No_8301,User_Id_87,Bike,2,Personal,22,2,38458.0,22,2,...,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549


In [86]:
# I will later use frequency encoding to handle UserID and Rider ID but let me remove it now

In [87]:
ridertrain.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                         'Arrival at Pickup - Day of Month','Arrival at Pickup - Weekday (Mo = 1)',
                         'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)','Arrival at Destination - Day of Month',
                         'Arrival at Destination - Weekday (Mo = 1)','Vehicle Type', 'Order No','Platform Type', 'User Id', 'Rider Id',
                         'Personal or Business',
                         'Precipitation in millimeters'], inplace=True)

In [88]:
ridertest.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,60250.0,27,3,...,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192,439,1511,13.3,171
1,Order_No_2699,User_Id_592,Bike,1,Personal,26,2,53001.0,26,2,...,NaN,-1.312745,36.875715,-1.288435,36.760677,Rider_Id_192,439,1511,13.3,171
2,Order_No_21486,User_Id_478,Bike,3,Business,29,5,42905.0,29,5,...,NaN,-1.255189,36.782203,-1.293393,36.790237,Rider_Id_192,439,1511,13.3,171
3,Order_No_19336,User_Id_2801,Bike,3,Business,2,1,33801.0,2,1,...,NaN,-1.310451,36.868724,-1.259542,36.787118,Rider_Id_192,439,1511,13.3,171
4,Order_No_20374,User_Id_3465,Bike,3,Business,14,4,46559.0,14,4,...,NaN,-1.333275,36.870815,-1.255189,36.782203,Rider_Id_192,439,1511,13.3,171


In [90]:
ridertest.drop(['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)',
                'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)','Vehicle Type','Platform Type','User Id', 'Rider Id',
                'Personal or Business',
                'Precipitation in millimeters'], axis=1, inplace=True)

In [91]:
ridertrain.head()

,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,Pickup - Time,Arrival at Destination - Time,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,9,5,34546.0,34810.0,36287.0,37650.0,38395.0,4,20.4,-1.317755,36.830370,-1.300406,36.829741,745,1637,1309,13.8,549
1,18,5,56477.0,56490.0,58036.0,58417.0,61303.0,20,24.5,-1.326774,36.787807,-1.356237,36.904295,2886,1637,1309,13.8,549
2,31,5,46301.0,47569.0,48002.0,48788.0,51403.0,6,24.7,-1.255189,36.782203,-1.273412,36.818206,2615,1637,1309,13.8,549
3,2,2,25930.0,25949.0,27324.0,27526.0,30512.0,18,15.2,-1.290315,36.757377,-1.223520,36.802061,2986,1637,1309,13.8,549
4,22,2,38458.0,38544.0,39360.0,39580.0,41182.0,7,19.2,-1.273524,36.799220,-1.300431,36.752427,1602,1637,1309,13.8,549


# Model Training

In [92]:
#Importing Neccessary Libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [93]:
# Splitting
Y = ridertrain['Time from Pickup to Arrival']
X = ridertrain.drop(columns='Time from Pickup to Arrival')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [94]:
rfr = RandomForestRegressor(n_estimators=180, max_depth=110,max_features=3, min_samples_leaf=3,min_samples_split=8,random_state=0)

In [95]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features=3, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=8,
                      min_weight_fraction_leaf=0.0, n_estimators=180,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [96]:
grid_param = {'n_estimators': [150, 160, 170, 180],
              'max_depth': [80, 90, 100, 110],
              'max_features': [2, 3],
              'min_samples_leaf': [3, 4, 5],
              'min_samples_split': [8, 10, 12],
             }

In [97]:
gscv = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=grid_param,
    scoring='neg_mean_absolute_error',
    cv=4,
    n_jobs=-1)

In [98]:
gscv.fit(X_train, y_train)

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [80, 90, 100, 110],


In [101]:
best_parameters = gscv.best_params_
best_parameters

{'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 170}

In [103]:
y_pred = rfr.predict(X_test)

In [112]:
mse = mean_squared_error(y_test,y_pred)
mse

494323.61409792386

In [111]:
rmse = mse**1/2
rmse

247161.80704896193

In [1]:
import folium

In [5]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [6]:
base_map = generateBaseMap()

In [7]:
base_map